In [1]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [2]:
# load_data
train_data = pd.read_csv('../input/train.csv')

labels = train_data.label.values.astype('int32')
train_data = train_data.drop('label', axis=1).as_matrix().astype('float32')

test_data = pd.read_csv('../input/test.csv').as_matrix().astype('float32')

# make label vectors
y_train = np_utils.to_categorical(labels) 

In [3]:
# normalize 0-1, max is 255, so...
X_train = train_data / np.max(train_data)
X_test = test_data / np.max(train_data)

# maybe norm by mean too?
# X_train = train_data / np.mean(X_train)
# X_test = test_data / np.mean(X_train)

In [4]:
img_size = (28, 28)

X_train = X_train.reshape(X_train.shape[0], img_size[0], img_size[1], 1)
X_test = X_test.reshape(X_test.shape[0], img_size[0], img_size[1], 1)
image_shape = (img_size[0], img_size[1], 1)

In [5]:

# number of convolutional filters to use
filters = 8
pool = (4, 4)
kernel = (3, 3)
dense = 128
dp = 0.2

model = Sequential()
model.add(Convolution2D(filters, 
                        kernel[0], 
                        kernel[1],
                        border_mode='same',
                        input_shape=image_shape))
model.add(Activation('relu'))
model.add(Convolution2D(filters, kernel[0], kernel[1]))
model.add(Activation('relu'))
model.add(Convolution2D(filters, kernel[0], kernel[1]))
model.add(Activation('relu'))
model.add(Convolution2D(filters, kernel[0], kernel[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool))
model.add(Dropout(dp))
model.add(Flatten())
model.add(Dense(dense))
model.add(Activation('relu'))
model.add(Dropout(dp))
model.add(Dense(y_train.shape[1]))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [6]:
print("Moneky see...")
model.fit(X_train, 
          y_train, 
          nb_epoch=10, 
          batch_size=32, 
          validation_split=0.1, 
          shuffle=True,
          verbose=1)

Moneky see...
Train on 37800 samples, validate on 4200 samples
Epoch 1/10
37800/37800 [==============================] - 71s - loss: 0.4838 - acc: 0.8422 - val_loss: 0.1389 - val_acc: 0.9571
Epoch 2/10
37800/37800 [==============================] - 70s - loss: 0.2110 - acc: 0.9348 - val_loss: 0.1011 - val_acc: 0.9664
Epoch 3/10
37800/37800 [==============================] - 74s - loss: 0.1720 - acc: 0.9466 - val_loss: 0.0955 - val_acc: 0.9695
Epoch 4/10
37800/37800 [==============================] - 73s - loss: 0.1451 - acc: 0.9552 - val_loss: 0.0716 - val_acc: 0.9771
Epoch 5/10
37800/37800 [==============================] - 70s - loss: 0.1307 - acc: 0.9594 - val_loss: 0.0666 - val_acc: 0.9783
Epoch 6/10
18144/37800 [=============>................] - ETA: 34s - loss: 0.1214 - acc: 0.9607

In [7]:
print("Monkey test...")
y_hat = model.predict_classes(X_train, verbose=0)
acc = sum(y_hat==labels)/len(labels)
best_so_far = 0.993119047619
print(acc, acc - best_so_far)

Monkey test...
0.986380952381 -0.00673809523805


In [8]:
print("Monkey do...")
predictions = model.predict_classes(X_test, verbose=0)

Monkey do...


In [9]:
save_me = pd.DataFrame({"Label": predictions})
save_me.index = save_me.index + 1

save_me.to_csv("keras_mnist_v0.1.csv",index_label='ImageId')

save_me.head()

,Label
1,2
2,0
3,9
4,9
5,3
